In [5]:
import pygame

from pygame.locals import (
    K_RIGHT,
    KEYDOWN,
    QUIT,
)
from taxi_service_model import TaxiService

In [6]:
v_corr = {}
g = {}
cnt = 0


def add_line_1(line_start, line_end, position):
    for i in range(line_start, line_end, 20):
        v_corr[f"{i} {position}"] = (i, position)
        g[f"{i} {position}"] = [f"{i-20} {position}", f"{i} {position-20}", f"{i+20} {position}", f"{i} {position+20}"]


def add_line_2(line_start, line_end, position):
    for i in range(line_start, line_end, 20):
        v_corr[f"{position} {i}"] = (position, i)
        g[f"{position} {i}"] = [f"{position-20} {i}", f"{position} {i-20}", f"{position+20} {i}", f"{position} {i+20}"]


add_line_1(20, 620, 20)
add_line_1(20, 620, 100)
add_line_1(20, 620, 200)
add_line_1(20, 160, 320)
add_line_1(160, 640, 420)

add_line_2(20, 320, 20)
add_line_2(20, 420, 160)
add_line_2(20, 420, 400)
add_line_2(20, 420, 620)

for x in g:
    lst_to_remove = []
    for i in range(4):
        if g[x][i] not in g:
            lst_to_remove.append(i)

    for i in lst_to_remove[::-1]:
        del g[x][i]

In [7]:
pygame.init()
# Parameters
w_width = 800
w_height = 800
# Set up the drawing window, adjust the size
screen = pygame.display.set_mode([w_width, w_height])

# Set background
screen.fill((255, 255, 255))

ts = TaxiService(g)


def draw(screen, v_cor):
    block_size = 20

    for i, j in v_cor.values():
        rect = pygame.Rect(i, j, block_size - 1, block_size - 1)
        pygame.draw.rect(screen, (210, 210, 210), rect, 0)

    for taxi in ts.taxis.values():
        i, j = v_corr[taxi.current_vertex]
        rect = pygame.Rect(i + 7, j + 7, block_size - 15, block_size - 15)
        pygame.draw.rect(screen, (210, 150, 10), rect, 0)

    for customer in ts.customers:
        i, j = v_corr[customer.current_vertex]
        rect = pygame.Rect(i + 2, j + 2, block_size - 15, block_size - 15)
        pygame.draw.rect(screen, (100, 150, 10), rect, 0)

    pygame.display.flip()


draw(screen, v_corr)
running = True

time_delay = 200  # 0.2 s
timer_event = pygame.USEREVENT + 1
pygame.time.set_timer(timer_event, time_delay)

while running:
    for event in pygame.event.get():
        if event.type == QUIT:
            running = False

        if event.type == KEYDOWN:
            if event.key == K_RIGHT:
                ts.make_step()
                draw(screen, v_corr)

pygame.quit()

KeyboardInterrupt: 